In [27]:
import pandas as pd
from gurobipy import Model, GRB

In [28]:
# Read the player data
players_df = pd.read_csv('/Users/mahinbindra/Downloads/BasketballPlayers.csv')

In [29]:
players_df['Average'] = players_df.iloc[:, 2:].mean(axis=1)

In [30]:
# Filter players with average skill above 2.05 (without resetting the index)
filtered_players_df = players_df[players_df['Average'] > 2.05]
filtered_players_df = filtered_players_df.drop(columns=['Average'])

In [31]:
# Number of players
num_players = len(filtered_players_df)

In [32]:
# Create the optimization model
model = Model("TrainingCampSelection")

In [33]:
# Create binary decision variables for each player (using original indices)
x = model.addVars(filtered_players_df.index, vtype=GRB.BINARY, name="Player")

In [34]:
# Pre-compute the player positions using original indices
guards = [i for i in filtered_players_df.index if filtered_players_df.loc[i, 'Position'] in ['G', 'G/F']]
forwards_centers = [i for i in filtered_players_df.index if filtered_players_df.loc[i, 'Position'] in ['F', 'C', 'F/C']]

In [35]:
# Total number of players selected
total_players_selected = sum(x[i] for i in filtered_players_df.index)

In [36]:
# At least 30% of the invitations should go to guards
model.addConstr(sum(x[i] for i in guards) >= 0.3 * total_players_selected, "Min_30_percent_guards")

<gurobi.Constr *Awaiting Model Update*>

In [37]:
# At least 40% of the invitations should go to forwards/centers
model.addConstr(sum(x[i] for i in forwards_centers) >= 0.4 * total_players_selected, "Min_40_percent_forwards_centers")

<gurobi.Constr *Awaiting Model Update*>

In [38]:
# Limit the total number of invitations to 21
model.addConstr(total_players_selected <= 21, "Total_Invitations_Limit")

<gurobi.Constr *Awaiting Model Update*>

In [39]:
# If any player from 20-24 (inclusive) is invited, all players from 72-78 (inclusive) cannot be, and vice versa
model.addConstr(sum(x[i] for i in filtered_players_df.index if 20 <= filtered_players_df.loc[i, 'Number'] <= 24) + sum(x[j] for j in filtered_players_df.index if 72 <= filtered_players_df.loc[j, 'Number'] <= 78) <= 1, "Group_20_24_vs_72_78")

<gurobi.Constr *Awaiting Model Update*>

In [40]:
# If any player from 105-114 (inclusive) is invited, at least one player from 45-49 (inclusive) and 65-69 (inclusive) must be invited
for i in [idx for idx in filtered_players_df.index if 105 <= filtered_players_df.loc[idx, 'Number'] <= 114]:
    model.addConstr(x[i] <= sum(x[j] for j in filtered_players_df.index if 45 <= filtered_players_df.loc[j, 'Number'] <= 49), f"Group_105_114_requires_45_49_for_{i}")
    model.addConstr(x[i] <= sum(x[k] for k in filtered_players_df.index if 65 <= filtered_players_df.loc[k, 'Number'] <= 69), f"Group_105_114_requires_65_69_for_{i}")

In [41]:
# At least one player must be invited from: 1-10, 11-20, 21-30, ..., 131-140, 141-150
for i in range(1, 151, 10):
    model.addConstr(sum(x[j] for j in filtered_players_df.index if i <= filtered_players_df.loc[j, 'Number'] < i + 10) >= 1, f"Group_{i}_{i+9}")

In [42]:
# Objective function to maximize total skill ratings
skills = ['Ball Handling', 'Shooting', 'Rebounding', 'Defense', 'Athletic Ability', 'Toughness', 'Mental Acuity']
model.setObjective(sum(filtered_players_df.loc[i, skill] * x[i] for i in filtered_players_df.index for skill in skills), GRB.MAXIMIZE)

In [43]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 29 rows, 65 columns and 295 nonzeros
Model fingerprint: 0xa42ff278
Variable types: 0 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 336.0000000
Presolve removed 1 rows and 12 columns
Presolve time: 0.00s
Presolved: 28 rows, 53 columns, 246 nonzeros
Variable types: 0 continuous, 53 integer (46 binary)
Found heuristic solution: objective 353.0000000

Root relaxation: objective 3.600000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

In [44]:
# Print the selected players and their details
selected_players = [i for i in filtered_players_df.index if x[i].X == 1]
count_guards = sum(1 for i in selected_players if i in guards)
count_forwards_centers = sum(1 for i in selected_players if i in forwards_centers)
total_selected = len(selected_players)
num_decision_variables = len(filtered_players_df)


print(f"Number of decision variables: {num_decision_variables}")

print("Selected players and their positions:")
for i in selected_players:
    print(f"Player {i}: {filtered_players_df.loc[i, 'Position']}")
print(f"Total guards selected: {count_guards}")
print(f"Total forwards/centers selected: {count_forwards_centers}")
print(f"Total players selected: {total_selected}")

# Print the value of the objective function
print(f"Value of objective function: {model.ObjVal}")

Number of decision variables: 65
Selected players and their positions:
Player 4: F/C
Player 6: F/C
Player 10: F
Player 25: G
Player 36: G/F
Player 46: G/F
Player 55: F/C
Player 66: G
Player 73: F/C
Player 89: F
Player 94: G/F
Player 103: G
Player 104: F/C
Player 109: G
Player 110: F
Player 117: G/F
Player 127: G/F
Player 131: G/F
Player 132: F
Player 133: F/C
Player 143: F
Total guards selected: 10
Total forwards/centers selected: 11
Total players selected: 21
Value of objective function: 360.0


In [45]:
import pandas as pd
from gurobipy import Model, GRB


print("\n####################################### PART H: FINDING MINIMUM INVITATIONS POSSIBLE #######################################################################\n")

# Read the player data
players_df = pd.read_csv('/Users/mahinbindra/Downloads/BasketballPlayers.csv')

players_df['Average'] = players_df.iloc[:, 2:].mean(axis=1)

# Filter players with average skill above 2.05 (without resetting the index)
filtered_players_df = players_df[players_df['Average'] > 2.05]
filtered_players_df = filtered_players_df.drop(columns=['Average'])

# Number of players
num_players = len(filtered_players_df)

# Create the optimization model
model = Model("TrainingCampSelection")

# Create binary decision variables for each player (using original indices)
x = model.addVars(filtered_players_df.index, vtype=GRB.BINARY, name="Player")

# Pre-compute the player positions using original indices
guards = [i for i in filtered_players_df.index if filtered_players_df.loc[i, 'Position'] in ['G', 'G/F']]
forwards_centers = [i for i in filtered_players_df.index if filtered_players_df.loc[i, 'Position'] in ['F', 'C', 'F/C']]

# Total number of players selected
total_players_selected = sum(x[i] for i in filtered_players_df.index)

# At least 30% of the invitations should go to guards
model.addConstr(sum(x[i] for i in guards) >= 0.3 * total_players_selected, "Min_30_percent_guards")

# At least 40% of the invitations should go to forwards/centers
model.addConstr(sum(x[i] for i in forwards_centers) >= 0.4 * total_players_selected, "Min_40_percent_forwards_centers")

# Limit the total number of invitations to 21
#model.addConstr(total_players_selected <= 21, "Total_Invitations_Limit")

# If any player from 20-24 (inclusive) is invited, all players from 72-78 (inclusive) cannot be, and vice versa
model.addConstr(sum(x[i] for i in filtered_players_df.index if 20 <= filtered_players_df.loc[i, 'Number'] <= 24) + sum(x[j] for j in filtered_players_df.index if 72 <= filtered_players_df.loc[j, 'Number'] <= 78) <= 1, "Group_20_24_vs_72_78")


# If any player from 105-114 (inclusive) is invited, at least one player from 45-49 (inclusive) and 65-69 (inclusive) must be invited
for i in [idx for idx in filtered_players_df.index if 105 <= filtered_players_df.loc[idx, 'Number'] <= 114]:
    model.addConstr(x[i] <= sum(x[j] for j in filtered_players_df.index if 45 <= filtered_players_df.loc[j, 'Number'] <= 49) + sum(x[k] for k in filtered_players_df.index if 65 <= filtered_players_df.loc[k, 'Number'] <= 69), f"Group_105_114_requires_{i}")


# At least one player must be invited from: 1-10, 11-20, 21-30, ..., 131-140, 141-150
for i in range(1, 151, 10):
    model.addConstr(sum(x[j] for j in filtered_players_df.index if i <= filtered_players_df.loc[j, 'Number'] < i + 10) >= 1, f"Group_{i}_{i+9}")


# Update the model
model.update()

# Print the number of constraints in the model
print("Number of constraints after adding:", len(model.getConstrs()))

# Check if there are any constraints in the model
constraints = model.getConstrs()
if not constraints:
    raise ValueError("No constraints found in the model")
last_feasible_solution = constraints[0]

# Change the objective function to minimize the total number of players selected
model.setObjective(total_players_selected, GRB.MINIMIZE)

# Find the smallest number of players that can be selected without causing infeasibility
min_players_selected = num_players
infeasible_constraint = None

for i in range(num_players, 0, -1):
    model.update()
    model.optimize()

    # Find the selected players
    selected_players = [i for i in filtered_players_df.index if x[i].X > 0.5]

# Print the details of the selected players
    print("Selected players:")
    for player in selected_players:
        player_data = filtered_players_df.loc[player]
        print(f"Player Number: {player_data['Number']}, Position: {player_data['Position']}")


    if model.status == GRB.INFEASIBLE:
        infeasible_constraint = model.getConstrs()[model.getConstrs().index(last_feasible_solution) + 1].ConstrName
        break
    elif model.status == GRB.OPTIMAL:
        min_players_selected = i
        # Update last_feasible_solution only if it's not the last constraint
        if model.getConstrs().index(last_feasible_solution) + 1 < len(model.getConstrs()):
            last_feasible_solution = model.getConstrs()[model.getConstrs().index(last_feasible_solution) + 1]


print(f"Value of the objective function (total number of players selected): {model.ObjVal}")
if infeasible_constraint:
    print(f"The constraint that caused infeasibility: {infeasible_constraint}")

print("\n####################################### FINDING WHAT CONSTRAINTS WOULD CAUSE INFEASIBILITY #######################################################################\n")

model.addConstr(total_players_selected <= model.ObjVal - 1, "Reduced_Player_Selection")

model.optimize()


if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Identifying the problematic constraint(s)...")
    model.computeIIS()
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Constraint causing infeasibility: {c.ConstrName}")


####################################### PART H: FINDING MINIMUM INVITATIONS POSSIBLE #######################################################################

Number of constraints after adding: 23
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23 rows, 65 columns and 225 nonzeros
Model fingerprint: 0xc2c165cb
Variable types: 0 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 15.0000000
Presolve removed 1 rows and 28 columns
Presolve time: 0.00s
Presolved: 22 rows, 37 columns, 135 nonzeros
Variable types: 0 continuous, 37 integer (23 binary)

Root relaxation: cutoff, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |    